In [19]:
def post_process(file):
    with open(file, "r", encoding="utf-8") as f:
        content = f.readlines()

    list_start = []
    for idx, line in enumerate(content):
        if "VIETTEL AI RACE" in line:
            list_start.append(idx)
    to_remove = set()
    for s in list_start:
        for j in (s, s + 1, s + 2):
            if 0 <= j < len(content):
                to_remove.add(j)

    content = [line for i, line in enumerate(content) if i not in to_remove]
    new_text = "\n".join(content)
    return new_text

In [21]:
import re
import pandas as pd
from io import StringIO

# --------------------------------------
# Normalize text within each table cell
# --------------------------------------
def normalize_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    # Convert <br> to spaces
    s = re.sub(r'<br\s*/?>', ' ', s, flags=re.IGNORECASE)
    # Remove multiple spaces
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

# ---------------------------------------------------------
# Automatically detect the header row of a markdown table
# ---------------------------------------------------------
def detect_header(lines: list):
    """
    Pick the header row based on:
    - First non-empty line containing multiple | delimeters
    """
    for i, line in enumerate(lines):
        if "|" in line and not re.match(r'^\s*\|\s*-', line):  # skip separator lines
            return i
    return 0


# ---------------------------------------------------------
# Convert markdown-like table to READOC-friendly HTML table
# ---------------------------------------------------------
def convert_markdown_to_html_table(md_table: str) -> str:
    # Normalize the incoming text lines
    lines = [l.strip() for l in md_table.strip().split("\n") if l.strip()]

    # Detect header row
    header_idx = detect_header(lines)
    header_line = lines[header_idx]
    # Clean header formatting (remove **bold**)
    clean_header = re.sub(r'\*{2}(.*?)\*{2}', r'\1', header_line)

    # Extract header names
    header_parts = [normalize_text(col) for col in clean_header.split("|") if col.strip()]

    # Extract body rows (skip header + separator)
    body_lines = [
        l for l in lines[header_idx + 1:]
        if "|" in l and not re.match(r'^\s*\|\s*-', l)
    ]

    # Build CSV-like temporary structure for pandas
    csv_str = ";;".join(header_parts) + "\n"
    for row in body_lines:
        row = row.replace("||", "^@")

        
        cols = [normalize_text(c.replace("^@", "||")) for c in row.split("|") if c.strip()]
        if len(cols) > len(header_parts):
            cols = cols[:len(header_parts)]
        csv_str += ";;".join(cols) + "\n"

    # Convert to DataFrame

    try:
        df = pd.read_csv(StringIO(csv_str), sep=";;")
    except Exception as e:
        print("Error parsing table:", e)
        print("header_parts:", header_parts)
        print(csv_str)

    # Final normalization
    for col in df.columns:
        df[col] = df[col].apply(normalize_text)

    # ---------------------------
    # Construct HTML output
    # ---------------------------
    html = [
        '<table>',
        '<colgroup><col/><col/><col/><col/></colgroup>',
        '<tbody>',
        '<tr>'
    ]

    # Headers
    for col in df.columns:
        html.append(f'<td><strong>{col}</strong></td>')
    html.append('</tr>')

    # Rows
    for _, row in df.iterrows():
        html.append('<tr>')
        for col in df.columns:
            html.append(f'<td>{row[col]}</td>')
        html.append('</tr>')

    html.append('</tbody></table>')

    return "\n".join(html)





In [9]:
def detect_table_and_convert_html(md):
    md_blocks = md.split("\n\n\n\n")
    md_blocks = [i for i in md_blocks if i.strip() != '']
    out_blocks = []
    for block in md_blocks:
        lines = [l.strip() for l in block.strip().split("\n") if l.strip()]
        first_line = lines[0]
        # consider it a table header only if it contains multiple pipes (>=3)
        if first_line.count("|") >= 3 and not re.match(r'^\s*\|\s*-', first_line):
            # print(block)
            html_output = convert_markdown_to_html_table(block)
            out_blocks.append(html_output)
        else:
            out_blocks.append(block)

    out_text = "\n".join(out_blocks)
    return out_text

    # print(f"Detected header line: {header_line}", header_idx)

In [22]:
import os
list_orin_file = "output/var_train_full"
list_post_file = list_orin_file+"_post"
os.makedirs(list_post_file, exist_ok=True)
for file in os.listdir(list_orin_file):
    if file.endswith(".md"):
        print(f"Processing file: {file}")
        orin_path = os.path.join(list_orin_file, file)
        post_path = os.path.join(list_post_file, file)
        md = post_process(orin_path)
        md = detect_table_and_convert_html(md)
        with open(post_path, "w", encoding="utf-8") as f:
            f.write(md)

Processing file: Public_045.md
Processing file: Public_165.md
Processing file: Public_175.md
Processing file: Public_026.md
Processing file: Public_264.md
Processing file: Public_107.md
Processing file: Public_040.md
Processing file: Public_584.md
Processing file: Public_257.md
Processing file: Public_576.md
Processing file: Public_567.md
Processing file: Public_263.md
Processing file: Public_628.md
Processing file: Public_210.md
Processing file: Public_089.md
Processing file: Public_158.md
Processing file: Public_051.md
Processing file: Public_215.md
Processing file: Public_114.md
Processing file: Public_618.md
Processing file: Public_279.md
Processing file: Public_106.md
Processing file: Public_503.md
Processing file: Public_011.md
Processing file: Public_282.md
Processing file: Public_060.md
Processing file: Public_186.md


/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1

Processing file: Public_041.md
Processing file: Public_194.md
Processing file: Public_115.md
Processing file: Public_625.md
Processing file: Public_059.md
Processing file: Public_095.md
Processing file: Public_187.md
Processing file: Public_631.md
Processing file: Public_526.md
Processing file: Public_565.md
Processing file: Public_273.md
Processing file: Public_184.md
Processing file: Public_523.md
Processing file: Public_605.md
Processing file: Public_209.md
Processing file: Public_583.md
Processing file: Public_079.md
Processing file: Public_078.md
Processing file: Public_008.md
Processing file: Public_515.md
Processing file: Public_586.md


/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1

Processing file: Public_065.md
Processing file: Public_179.md
Processing file: Public_166.md
Processing file: Public_020.md
Processing file: Public_254.md
Processing file: Public_047.md
Processing file: Public_614.md
Processing file: Public_199.md
Processing file: Public_098.md
Processing file: Public_151.md
Processing file: Public_066.md
Processing file: Public_501.md
Processing file: Public_593.md
Processing file: Public_068.md
Processing file: Public_094.md
Processing file: Public_575.md
Processing file: Public_097.md
Processing file: Public_076.md
Processing file: Public_574.md
Processing file: Public_080.md
Processing file: Public_570.md
Processing file: Public_072.md
Processing file: Public_594.md
Processing file: Public_616.md
Processing file: Public_280.md
Processing file: Public_522.md
Processing file: Public_520.md
Processing file: Public_496.md
Processing file: Public_122.md
Processing file: Public_028.md
Processing file: Public_001.md
Processing file: Public_266.md
Processi

...

/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1

Processing file: Public_057.md
Processing file: Public_038.md
Processing file: Public_276.md
Processing file: Public_061.md
Processing file: Public_478.md
Processing file: Public_156.md
Processing file: Public_120.md
Processing file: Public_278.md
Processing file: Public_579.md
Processing file: Public_192.md
Processing file: Public_610.md
Processing file: Public_582.md
Processing file: Public_255.md


/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1

Processing file: Public_081.md
Processing file: Public_174.md
Processing file: Public_073.md
Processing file: Public_118.md
Processing file: Public_157.md
Processing file: Public_006.md
Processing file: Public_514.md
Processing file: Public_185.md
Processing file: Public_568.md
Processing file: Public_033.md
Processing file: Public_058.md
Processing file: Public_633.md


/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1010720/3195515709.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(csv_str), sep=";;")
/tmp/ipykernel_1